In [1]:
import os
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
import random

import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from PIL import Image

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPool2D, GlobalAvgPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop, Adam

In [2]:
data = (r'C:\Users\Deepak L\Music\FAkE CURRNCY\DATASET\Testing')

In [3]:
images=[]
import os
for dirname,_,filenames in os.walk(data):
  for filename in filenames:
    file_name, file_extension = os.path.splitext(filename)
    if file_extension == '.db':
      continue
    else:
      img=os.path.join(dirname,filename)
      images.append(img)

In [4]:
images[:10]

['C:\\Users\\Deepak L\\Music\\FAkE CURRNCY\\DATASET\\Testing\\Fake.jpeg',
 'C:\\Users\\Deepak L\\Music\\FAkE CURRNCY\\DATASET\\Testing\\Real.jpg']

In [5]:
len(images)

2

In [6]:
class_values = []
for i in images:
    j = i.split('\\')
    class_values.append(j[-2])

In [7]:
class_values[:10]

['Testing', 'Testing']

In [8]:
len(class_values)

2

In [9]:
temp = list(zip(images,class_values))
random.shuffle(temp)
images, class_values = zip(*temp)
data = pd.DataFrame(list(zip(images, class_values)), columns=['image_path', 'class_label'])

In [10]:
data

,image_path,class_label
0,C:\Users\Deepak L\Music\FAkE CURRNCY\DATASET\T...,Testing
1,C:\Users\Deepak L\Music\FAkE CURRNCY\DATASET\T...,Testing


In [11]:
data.class_label.value_counts()

class_label
Testing    2
Name: count, dtype: int64

In [12]:
df_c0 = data[data['class_label'] =="Fake"]
df_c1 = data[data['class_label'] =="Real"]

In [13]:
unsampled_df_c0 = resample(df_c0, replace=True, random_state=23, n_samples=200)
unsampled_df_c1 = resample(df_c1, replace=True, random_state=23, n_samples=200)

ValueError: high <= 0

In [ ]:
df_unsampled = pd.concat([unsampled_df_c0, unsampled_df_c1])
df_unsampled

In [ ]:
df_unsampled.head()

In [ ]:
df_unsampled.tail()

In [ ]:
df_unsampled.class_label.value_counts()

In [ ]:
df_unsampled.shape

In [ ]:
data = df_unsampled.sample(frac=1)
data.shape

In [ ]:
data[:10]

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.countplot(x="class_label", data=df_unsampled, palette=sns.color_palette("cubehelix", 4))
plt.xticks(rotation=90)
plt.title("Class Label Counts", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("count", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("Class_label", {"fontname": "serif", "fontweight":"bold"})

In [ ]:
def resize_image(img):
    file = cv2.imread(img)
    resized = cv2.resize(file, (224,224), interpolation = cv2.INTER_AREA)
    resized= np.array(resized)
    image = np.reshape(resized,[1,224,224,3])
    return resized

In [ ]:
def resize_image(img):
    file = cv2.imread(img)
    print("START")
    print(file)
    print(img)
    print("END")
    resized = cv2.resize(file, (224,224), interpolation = cv2.INTER_AREA)
    resized= np.array(resized)
    image = np.reshape(resized,[1,224,224,3])
    return resized

In [ ]:
images=[resize_image(img) for img in data['image_path']]

In [ ]:
images = []
for i in data['image_path']:
    file_name, file_extension = os.path.splitext(i)
    if file_extension == '.db':
        print(file_name)
    else:
        app = resize_image(i)
        images.append(app)

In [ ]:
len(images)

In [ ]:
data['class_label'][:15]

In [ ]:
class_names = list(data['class_label'])

In [ ]:
images[0].shape

In [ ]:
plt.imshow(images[86])
plt.title(class_names[86])

In [ ]:
plt.imshow(images[188])
plt.title(class_names[188])

In [ ]:
images = np.array(images)

In [ ]:
images.shape

In [ ]:
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
y = enc.fit_transform(data['class_label'])

In [ ]:
enc.classes_

In [ ]:
data['class_label'][:10]

In [ ]:
y[:10]

In [ ]:
y = []
for i in data['class_label']:
    if i == 'Real':
        k= [1, 0]
        y.append(k)
    else:
        k = [0, 1]
        y.append(k)

In [ ]:
y[:10]

In [ ]:
y=np.array(y)
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, y, test_size=0.3, stratify=y, random_state=43)

In [ ]:
y_train[:5]

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
batch_size = None

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', batch_input_shape = (batch_size,224, 224, 3)))


model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.4))



model.add(GlobalMaxPooling2D())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
img, x_train, y_train

In [ ]:
his = model.fit(x_train, y_train, epochs=20, batch_size=25)

In [ ]:
score_1 = model.evaluate(x_train, y_train, verbose=0)
print("Train Accuracy : ", score_1[1])

In [ ]:
score_1 = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy : ", score_1[1])

In [ ]:
results = pd.DataFrame(columns=['model', 'test_accuracy'])
new = ['CNN', score_1[1]]
results.loc[0] = new

In [ ]:
model.save(r'C:\Users\Deepak L\Music\FAkE CURRNCY\CODE\FRONT END\frontend\CNN_model_n.h5')

In [ ]:
results

In [ ]:
for i in range(20):
    plt.figure(figsize=(15, 15))
    plt.subplot(4, 5, i+1)
    pred = np.argmax(model.predict(np.array([x_train[i]]))[0])
    print(x_train[i])
    act = np.argmax(y_test[i])
    plt.title("Predicted Class : {}\n Actual Class : {}".format(enc.classes_[pred], enc.classes_[act]))
    plt.imshow(x_test[i])
act = np.argmax(y_test[i])
print("Predicted class : {}", format(enc.classes_[pred]))
print("Actual Class : {}", format(enc.classes_[act]))

plt.imshow(x_test[i])

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [ ]:
vgg.trainable = False

In [ ]:
model_1 = Sequential()
model_1.add(vgg)
model_1.add(Flatten())
model_1.add(Dense(128, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(2, activation='softmax'))

In [ ]:
opt = Adam(lr=0.001)
optimizer = tf.keras.optimizers.legacy.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model_1.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
his2= model_1.fit(x_train, y_train, epochs = 10, validation_data = (x_test,y_test),batch_size = 28)

In [ ]:
score_3 = model_1.evaluate(x_train, y_train, verbose=0)
print('train accuracy:', score_3[1])

In [ ]:
score_4 = model_1.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score_4[1])

In [ ]:
model.save(r'C:\Users\Deepak L\Music\FAkE CURRNCY\CODE\FRONT END\frontend\VGG_model_n.h5')

In [ ]:
for i in range(20):
  plt.figure(figsize=(15,15))
  plt.subplot(4,5,i+1)
  pred = np.argmax(model_1.predict(np.array([x_test[i]]))[0])
  act = np.argmax(y_test[i])
  plt.title("Predicted class: {}\n Actual class: {}".format(enc.classes_[pred],enc.classes_[act]))
  plt.imshow(x_test[i])

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(4,5,i+1)
pred = np.argmax(model_1.predict(np.array([x_test[0]]))[0])
act = np.argmax(y_test[0])
plt.title("Predicted class: {}\n Actual class: {}".format(enc.classes_[pred],enc.classes_[act]))
plt.imshow(x_test[0])

In [ ]:
results = pd.DataFrame(columns=['model', 'test_accuracy'])
new = ['CNN', score_1[1]]
results.loc[0] = new

In [ ]:
new = ['VGG', score_4[1]]
results.loc[1] = new

In [ ]:
results